In [2]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.autograd import Variable
import numpy as np
from solvers.OMISTL import OMISTL
import cvxpy as cp
from utils import *
import pickle, os
from MPC_prob import MPC

from tqdm import tqdm
from para import paraset
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'#不设这个解不出#

C:\Users\msi\anaconda3\envs\coco\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\msi\anaconda3\envs\coco\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\msi\anaconda3\envs\coco\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


## Different horizon for training

In [3]:
N= 6
paraset(N=N,n_obs=5,Qs=1,Rs=0,num_probs=20000)

In [4]:
#pass the value from config to dict and para
relative_path = os.getcwd()
dataset_name = 'MPC_horizon_{}'.format(N)
# dataset_name = 'NSTL_{}_horizon_{}'.format(Qs, N)
config_fn = os.path.join(relative_path, 'config', dataset_name+'.p')

# config = [dataset_name, [prob_params] ,sampled_params]
outfile = open(config_fn,"rb")
config=pickle.load(outfile)
outfile.close()

train_fn = 'train_horizon_{}.p'.format(N)
train_fn = os.path.join(relative_path, 'data', dataset_name, train_fn)
test_fn = 'test_horizon_{}.p'.format(N)
test_fn = os.path.join(relative_path, 'data', dataset_name, test_fn)

all_params = ['N', 'Ak', 'Bk', 'Q', 'R', 'n_obs', \
            'posmin', 'posmax', 'velmin', 'velmax', \
            'umin', 'umax']

param_dict={}
i=0
len_para = len(all_params)
for param in all_params:
    param_dict[param]= config[1][i]
    i+=1
    if i == len_para:
        break

N = param_dict['N']
Ak = param_dict['Ak']
Bk = param_dict['Bk']
Q = param_dict['Q']
R = param_dict['R']
n_obs = param_dict['n_obs']
umin = param_dict['umin']
umax = param_dict['umax']
sampled_params = config[2]
n_obs = config[3]
num_probs = config[4]
border_size = config[5]
box_buffer = config[6]
min_box_size = config[7]
max_box_size = config[8]
posmin = config[9]
posmax = config[10]
velmin = config[11]
velmax = config[12]
n = config[13]
m= config[14]

obs_fix = config[15]
xg_fix = config[16]
if obs_fix:
    obstacles = config[-1]

config_fn = os.path.join(relative_path, 'config', dataset_name+'.p')#

prob = MPC(config=config_fn)
#create numpy containers for data: (params, x, u, y, J*, solve_time)
params = {}
if 'x0' in sampled_params:
    params['x0'] = np.zeros((num_probs,2*n))
if 'xg' in sampled_params:
    params['xg'] = np.zeros((num_probs,2*n))
if 'obstacles' in sampled_params:
    params['obstacles'] = np.zeros((num_probs, 4, n_obs))

X = np.zeros((num_probs, 2*n, N));
U = np.zeros((num_probs, m, N-1))
Y = np.zeros((num_probs, 4*n_obs, N-1)).astype(int)
Z = np.zeros((num_probs, 2*n_obs, N-1)).astype(int)

costs = np.zeros(num_probs)
solve_times = np.zeros(num_probs)

prob.sampled_params = ['x0', 'xg', 'obstacles']

#solving MICP
ii_toggle = 0
obs_new_ct = 5
ii=0
obstacles = config[-1]

if obs_fix:
    for ii in tqdm(range(num_probs)):
        x0 = findIC(obstacles, posmin, posmax, velmin, velmax)
        params['obstacles'][ii,:] = np.reshape(np.concatenate(obstacles, axis=0), (n_obs,4)).T
        p_dict = {}
        params['x0'][ii,:] = x0
        xg= findIC(obstacles, posmin, posmax, velmin, velmax)
        params['xg'][ii,:] = xg

        p_dict['x0'] = params['x0'][ii,:]
        p_dict['xg'] = params['xg'][ii,:]
        p_dict['obstacles'] = params['obstacles'][ii,:]

        prob_success = False
        try:
            # with time_limit(20):
            prob_success, cost, solve_time, optvals = prob.solve_stl(p_dict, solver=cp.GUROBI)
        except (KeyboardInterrupt, SystemExit):
            raise
        except:
            print('solver failed at '.format(ii))

        if prob_success:
            costs[ii] = cost; solve_times[ii] = solve_time
            X[ii,:,:], U[ii,:,:], Y[ii,:,:], Z[ii,:,:] = optvals
            ii += 1
else:
    print('choose to fix obstalce')

## shuffle the data because of the spatial orders
num_train = int(num_probs*0.9)
arr = np.arange(num_probs)
np.random.shuffle(arr)

if 'x0' in sampled_params:
    params['x0'] = params['x0'][arr]
if 'xg' in sampled_params:
    params['xg'] = params['xg'][arr]
if 'obstacles' in sampled_params:
    params['obstacles'] = params['obstacles'][arr]

costs = costs[arr]
solve_times = solve_times[arr]


X = X[arr]
U = U[arr]
Y = Y[arr]
Z = Z[arr]

train_params = {}; test_params = {}
if 'x0' in sampled_params:
    train_params['x0'] = params['x0'][:num_train,:]
    test_params['x0'] = params['x0'][num_train:,:]
if 'obstacles' in sampled_params:
    train_params['obstacles'] = params['obstacles'][:num_train,:]
    test_params['obstacles'] = params['obstacles'][num_train:,:]
if 'xg' in sampled_params:
    train_params['xg'] = params['xg'][:num_train,:]
    test_params['xg'] = params['xg'][num_train:,:]

train_data = [train_params]
train_data += [X[:num_train,:,:], U[:num_train,:,:], Y[:num_train,:,:],Z[:num_train,:,:]]
train_data += [costs[:num_train], solve_times[:num_train]]

test_data = [test_params]
test_data += [X[num_train:,:,:], U[num_train:,:,:], Y[num_train:,:,:], Z[:num_train,:,:]]
test_data += [costs[num_train:], solve_times[num_train:]]

train_file = open(train_fn,'wb')
pickle.dump(train_data,train_file); train_file.close()

test_file = open(test_fn, 'wb')
pickle.dump(test_data,test_file); test_file.close()

In [5]:
relative_path = os.getcwd()
dataset_name = 'MPC_horizon_{}'.format(N)
# dataset_name = 'NSTL_{}_horizon_{}'.format(Qs, N)
config_fn = os.path.join(relative_path, 'config', dataset_name+'.p')
prob = MPC(config=config_fn) #use default config, pass different config file oth.

relative_path = os.getcwd()
dataset_fn = relative_path + '/data/' + dataset_name

##load train data
train_file = open(dataset_fn+'/train_horizon_{}.p'.format(N),'rb')
# train_file = open(dataset_fn+'/train.p','rb')
train_data = pickle.load(train_file)
p_train, x_train, u_train, y_train, z_train, cost_train, times_train = train_data
train_file.close()

x_train = train_data[1] #X sequence
y_train = train_data[3] #Y sequence

##load test data
test_file = open(dataset_fn+'/test_horizon_{}.p'.format(N),'rb')
# test_file = open(dataset_fn+'/test.p','rb')
# p_test, x_test, u_test, y_test, c_test, times_test = pickle.load(test_file)
test_data = pickle.load(test_file)
p_test, x_test, u_test, y_test,z_test, cost_test,times_test = test_data
test_file.close()

In [6]:
system = 'MPC'
prob_features = ['x0','xg','obstacles']
# prob_features = ['x0','obstacles_map']

MPC_obj = OMISTL(system, prob, prob_features)

n_features = 33
MPC_obj.construct_strategies(n_features, train_data)
print(MPC_obj.n_strategies)
MPC_obj.n_evals
MPC_obj.setup_network()

fn_saved = 'D:\Curious\OMISTL\\models\\MPC_horizon_{}.pt'.format(N)
MPC_obj.load_network(fn_saved)

231
Loading presaved classifier model from D:\Curious\OMISTL\models\MPC_horizon_6.pt


In [7]:
# MPC_obj.training_params['TRAINING_ITERATIONS'] = 500
# MPC_obj.train(train_data=train_data, verbose=True)
# MPC_obj.model

In [11]:
outfile = open(config_fn,"rb")
config=pickle.load(outfile)
velmin = -0.2
velmax = 0.2
posmin = np.zeros(2)
n_obs = config[1][5]

ft2m = 0.3048
posmax = ft2m*np.array([12.,9.])
max_box_size = 0.75
min_box_size = 0.25
box_buffer = 0.025
border_size = 0.05

obstacles = config[-1]


# x0 = findIC(obstacles, posmin=posmin, posmax=posmax, velmin=velmin, velmax=velmax)
# xg = findIC(obstacles, posmin=posmin, posmax=posmax, velmin=velmin, velmax=velmax)
x0 = [0.1,0.1,0,0]
xg = [3,0.5,0,0]

# obstacles = \
#         random_obs(n_obs, posmin, posmax, border_size, box_buffer, min_box_size, max_box_size)
obstacles= config[-1]

prob_success = False
while not prob_success:
    prob_params = {}
    idx = np.random.randint(test_data[1].shape[0])
    for k in p_test.keys():
        prob_params[k] = p_test[k][idx]
    # prob_params['obstacles'] =  np.reshape(np.concatenate(obstacles, axis=0), (n_obs,4)).T
    # prob_params['x0'] = x0
    # prob_params['xg'] = xg

    prob_success, cost, total_time, n_evals, optvals,y_guess = MPC_obj.forward(prob_params, solver=cp.GUROBI, max_evals=10)
    # prob_success, cost, total_time, n_evals, optvals = coco_obj.forward(p_dict, solver=cp.GUROBI, max_evals=10)
    if prob_success:
        print('Found solution! ' + "n_evals = " + str(n_evals))
    else:
        print('Failed')
total_time

Found solution! n_evals = 1


0.00700688362121582

In [81]:
n_test = 2000
framework = 'OMISTL'
n_succ = 0
count = 0
gurobi_fail = 0

costs = []
total_time_ML = []
num_solves = []

cost_ratios = []
costs_ip = []
total_time_ip = []

for ii in tqdm(range(n_test)):
    if ii % 1000 == 0:
        print('{} / {}'.format(ii,n_test))
    prob_params = {}
    for k in p_test.keys():
        prob_params[k] = p_test[k][ii]

    try:
        prob_success, cost, total_time, n_evals, optvals,y_guess = MPC_obj.forward(prob_params, max_evals=10, solver=cp.MOSEK)

        if prob_success:
            n_succ += 1
            costs += [cost]
            total_time_ML += [total_time]
            num_solves += [n_evals]

            true_cost = cost_test[ii]
            costs_ip += [true_cost]
            total_time_ip += [times_test[ii]]

            cost_ratios += [cost / true_cost]
        count += 1
    except (KeyboardInterrupt, SystemExit):
        raise
    except:
        print('Solver failed at {}'.format(ii))
        gurobi_fail += 1
        continue

costs = np.array(costs)
cost_ratios = np.array(cost_ratios)
total_time_ML = np.array(total_time_ML)
num_solves = np.array(num_solves, dtype=int)

costs_ip = np.array(costs_ip)
total_time_ip = np.array(total_time_ip)

percentage = 100 * float(n_succ) / float(count)
dict = {'framework':framework, 'costs':costs, 'total_time_ML':total_time_ML,'num_solves':num_solves, 'costs_ip':costs_ip, 'total_time_ip':total_time_ip, 'cost_ratios':cost_ratios,'percentage':percentage}
f_save = open(dataset_fn+'/result.pkl', 'wb')
pickle.dump(dict, f_save)
f_save.close()

  0%|          | 0/200 [00:00<?, ?it/s]

0 / 200


 75%|███████▌  | 150/200 [00:06<00:01, 28.98it/s]

Solver failed at 147


100%|██████████| 200/200 [00:08<00:00, 24.33it/s]


In [83]:
# load results
f_read = open(dataset_fn+'/result.pkl', 'rb')
results = pickle.load(f_read)
f_read.close()

{'framework': 'OMISTL',
 'costs': array([  6.79233254,  18.13777287,  38.83775711,  80.83986806,
         73.03448054,  10.97165293,  53.98622899,  16.05846819,
         21.88866498,   3.1546086 ,   0.74165639,  33.01794866,
         36.94406138,   5.15449137,   7.31219968,  23.44958828,
         32.98543018,   6.30914069,   2.16753224,  22.24978338,
         62.74683114, 157.63531686,   0.75325112,  20.97936187,
         98.43459508,  26.89001367,   9.76716558,   7.4260475 ,
          8.17205545,   4.40406445,   0.97879872,   8.72282205,
         60.8712056 ,  82.36706631,  66.18280361,  50.43168996,
         58.06034436,  14.31410477,  92.38008208,  41.44742038,
         32.77146528,   3.85774677,   3.2442994 ,   3.58064177,
         34.82965695,   7.89805972,  22.95853047,  22.68586629,
         32.10057746,  31.43960443,  14.94823928,  54.08014613,
          7.09475635,   0.54631727,  14.06000322,  41.55167178,
         50.95411591,  84.18413151,  14.0182002 ,  19.12721898,
       

In [ ]:
from test_results import test_strategy
test_strategy(7,n_obs=5)

  1%|          | 207/20000 [00:24<53:07,  6.21it/s] 

In [ ]:
test_strategy(7,n_obs=5)